In [8]:
import cv2
import pprint
import requests

import matplotlib.pyplot as plt
import numpy as np

from azure_cfg import api_key
from skimage import io

%matplotlib inline

In [2]:
subscription_key = api_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze?"

image_url = "https://oxfordportal.blob.core.windows.net/vision/Analysis/3.jpg"

In [3]:
def extract_high_level_features(image_url):
    headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
    params   = {'visualFeatures': 'Adult,Categories,Description,Color,Faces,ImageType,Tags'}
    data     = {'url': image_url}

    response = requests.post(vision_analyze_url, headers=headers, params=params, json=data)
    response.raise_for_status()
    data = response.json()
    
    return {
        "clip_art_type"                      : data["imageType"]["clipArtType"],
        "line_drawing_type"                  : data["imageType"]["lineDrawingType"],
        "is_black_and_white"                 : int(data["color"]["isBwImg"]),
        "is_adult_content"                   : int(data["adult"]["isAdultContent"]),
        "adult_score"                        : data["adult"]["adultScore"],
        "is_racy"                            : int(data["adult"]["isRacyContent"]),
        "racy_score"                         : data["adult"]["racyScore"],
        "has_faces"                          : int(len(data["faces"])),
        "num_faces"                          : len(data["faces"]),
        "is_dominant_color_background_black" : int(data["color"]["dominantColorBackground"] == "Black"),
        "is_dominant_color_foreground_black" : int(data["color"]["dominantColorForeground"] == "Black")
    }

In [4]:
high_level_features = extract_high_level_features(image_url)
pprint.pprint(high_level_features)

{'adult_score': 0.013014059513807297,
 'clip_art_type': 0,
 'has_faces': 0,
 'is_adult_content': 0,
 'is_black_and_white': 0,
 'is_dominant_color_background_black': 1,
 'is_dominant_color_foreground_black': 1,
 'is_racy': 0,
 'line_drawing_type': 0,
 'num_faces': 0,
 'racy_score': 0.01406136155128479}


In [5]:
width, height = 128, 64
def extract_low_level_features(image_url, descriptor):
    image = io.imread(image_url)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (width,height))

    hog = descriptor.compute(image)
    hog = hog.flatten()
    return hog

In [6]:
descriptor = cv2.HOGDescriptor(_winSize = (width,height),
                               _blockSize = (16,16),
                               _blockStride = (8,8),
                               _cellSize = (8,8),
                               _nbins = 9)

low_level_features = extract_low_level_features(image_url, descriptor)
print(low_level_features)

[0.07069091 0.07552709 0.15499128 ... 0.19493683 0.12356531 0.0914105 ]


In [7]:
from pandas import DataFrame
from IPython.display import display

features = np.array(list(high_level_features.values()))
features = np.append(features, low_level_features)

df = DataFrame(data=np.reshape(features, (1,len(features))))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,3781,3782,3783,3784,3785,3786,3787,3788,3789,3790
0,0.0,0.0,1.0,0.0,0.014061,0.0,0.0,0.013014,1.0,0.0,...,0.018814,0.065769,0.004625,0.089652,0.194971,0.336087,0.197564,0.194937,0.123565,0.09141
